In [4]:
import src
from keplergl import KeplerGl
import geopandas as gpd

In [30]:
cities = ['Aguascalientes','Cuernavaca','Guadalajara','Laguna','Merida',
          'Mexicali','Mexico','Monterrey','Puebla','Querétaro','Saltillo',
         'SLP','Toluca']

In [31]:
map_1 = KeplerGl(height=800)
for city in cities:
    gdf = gpd.read_file(f'../data/processed/{city}_hex_bins.geojson')
    gdf['dist_promedio'] = (gdf['dist_farmacias']+gdf['dist_supermercados'])/2
    map_1.add_data(gdf[gdf['dist_promedio']>0], city)
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Aguascalientes': {'index': [0, 2, 6, 14, 19, 22, 25, 26, 30, 37, 40, 45, 48, 50, 54, 57, 59, 6…

In [61]:
# Save map_1 config to a file
with open(f'config_kepler_plot.py', 'w') as f:
   f.write('config = {}'.format(map_1.config))

In [62]:
# load the config
%run config_kepler_plot.py

In [64]:
map_1.save_to_html(file_name=f'../output/html/Mexico_accesibilidad.html', config=config,read_only=True)

Map saved to ../output/html/Mexico_accesibilidad.html!


In [59]:
for i in range(13):
    map_1.config['config']['visState']['layers'][i]['config']['visConfig']['opacity']=0.1

In [63]:
config['config']['visState']['layers'][12]

{'id': 'owtynpq',
 'type': 'geojson',
 'config': {'dataId': 'Toluca',
  'label': 'Toluca',
  'color': [34, 63, 154],
  'columns': {'geojson': 'geometry'},
  'isVisible': True,
  'visConfig': {'opacity': 0.1,
   'strokeOpacity': 0.8,
   'thickness': 0.5,
   'strokeColor': [218, 112, 191],
   'colorRange': {'name': 'UberPool',
    'type': 'diverging',
    'category': 'Uber',
    'colors': ['#FAE300',
     '#FAC200',
     '#FD7900',
     '#E31A1A',
     '#CF1750',
     '#AE0E7F',
     '#7A0DA6',
     '#482BBD',
     '#2C51BE',
     '#223F9A'],
    'reversed': True},
   'strokeColorRange': {'name': 'UberPool',
    'type': 'diverging',
    'category': 'Uber',
    'colors': ['#FAE300',
     '#FAC200',
     '#FD7900',
     '#E31A1A',
     '#CF1750',
     '#AE0E7F',
     '#7A0DA6',
     '#482BBD',
     '#2C51BE',
     '#223F9A'],
    'reversed': True},
   'radius': 10,
   'sizeRange': [0, 10],
   'radiusRange': [0, 50],
   'heightRange': [0, 500],
   'elevationScale': 5,
   'stroked': True,
  